In [1]:
# librerías
import os
import pickle
import pandas as pd
import numpy as np
import gower
from sklearn.svm import SVC
from sklearn.model_selection import  GridSearchCV, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, classification_report
from sklearn.manifold import MDS
import gc

In [2]:
# directorios
dataset_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(''))), 'datasets')
image_path = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(''))), 'images')

In [15]:
# dataset b (sin llamante_edad y con casos completos de victima_edad)

llamados_v2= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v2.xlsx'))

In [16]:
# quitar llamante edad
llamados_v2.drop(['llamante_edad'], axis=1, inplace=True) 

In [17]:
dataset_b = llamados_v2[~llamados_v2['victima_edad'].isnull()]

In [18]:

dataset_b.loc[:, 'victima_convive_agresor'] = dataset_b['victima_convive_agresor'].replace({'NS/NC': pd.NA})


In [19]:
# mapear SI NO a 1 0
dataset_b['victima_convive_agresor'] = dataset_b['victima_convive_agresor'].map({'SI': 1, 'NO': 0})


/var/folders/gm/4r6lhs5j5p12g3_ml4zjfy500000gq/T/ipykernel_3359/287204785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_b['victima_convive_agresor'] = dataset_b['victima_convive_agresor'].map({'SI': 1, 'NO': 0})


In [20]:
# chequear proporciones previas
print('prop. SI:', len(dataset_b[dataset_b['victima_convive_agresor']==1])/len(dataset_b)*100)
print('prop. NO:',len(dataset_b[dataset_b['victima_convive_agresor']==0])/len(dataset_b)*100)

prop. SI: 15.15958987429763
prop. NO: 67.59543532410358


In [21]:
dataset_b.loc[:,'victima_convive_agresor'].isna().sum()

2977

In [22]:
# separar features de target
X = dataset_b.drop(['victima_convive_agresor'], axis=1)


In [23]:
y_previo = dataset_b['victima_convive_agresor']

In [24]:
# guardar los índices de casos NSNC (vacíos) y sacaros del target (son solo para el test final)
nsnc_indices = y_previo[y_previo.isna()].index


In [13]:
#y = y_previo.drop(nsnc_indices)

In [25]:

# Apply Gower distance on the entire dataset
gower_X = gower.gower_matrix(X)
print("Gower transformation completed.")

# Run NMDS on the entire dataset
lista_dimensiones = [2, 3, 4, 5, 6, 7, 20]
lista_modelos = []
contador_exp=0
for i in lista_dimensiones:
    print(f'n_components: {i}')
    nmds = MDS(n_components=2, metric=False, dissimilarity='precomputed', max_iter=300, random_state=0, normalized_stress=True)
    X_nmds = nmds.fit_transform(gower_X)
    print('NMDS fit_transform completed.')

    # Use integer positions for nsnc_indices to avoid misalignment issues
    nsnc_pos = np.where(y_previo.isna())[0]  # Get the positions of NSNC rows

    # Extract the test set (rows corresponding to NSNC values)
    test_final = X_nmds[nsnc_pos, :]  # Use integer-based indexing
    print('Test final created')
    
    # Store the test set in a DataFrame
    test_final_df = pd.DataFrame(test_final, columns=[f'Component_{j}' for j in range(test_final.shape[1])])



    # Remove NSNC indices from the transformed dataset for training
    X_nmds_clean = pd.DataFrame(X_nmds).drop(nsnc_pos).reset_index(drop=True)
    y = y_previo.drop(nsnc_pos).reset_index(drop=True)

    print('Training SVM model...')
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=2)
    for train_index, test_index in sss.split(X_nmds_clean, y):
        X_train, X_test = X_nmds_clean.iloc[train_index], X_nmds_clean.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Parámetros de gridsearch
    param_grid = {
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto']
    }


    svm = SVC(class_weight='balanced')


    # GridSearchCV
    grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # Mejor modelo
    best_model = grid_search.best_estimator_

    # Aplicar al test set
    y_pred = best_model.predict(X_test)

   
   
    lista_modelos.append({
        'Experimento': contador_exp,
        'N Components': i,
        'Mejores parámetros': grid_search.best_params_,
        'Mejor acc. en cross validation': grid_search.best_score_,
        'Test set acc.': accuracy_score(y_test, y_pred),
        'Classification report': classification_report(y_test, y_pred)
    })

    print(f'Stress for {i} components: {round(nmds.stress_, 2)}')
    contador_exp += 1



Gower transformation completed.
n_components: 2
NMDS fit_transform completed.
Test final created


KeyError: '[11, 49, 171, 186, 254, 258, 264, 266, 268, 351, 352, 363, 406, 446, 447, 492, 552, 657, 721, 729, 927, 953, 1003, 1012, 1055, 1091, 1107, 1448, 1481, 1498, 2016, 2122, 2592, 2712, 2805, 2808, 2866, 3227, 3330, 3507, 3680, 3712, 3713, 3734, 3980, 3982, 4713, 4715, 4718, 4758, 4801, 4828, 4829, 4830, 5082, 5126, 5220, 5393, 5394, 5588, 5921, 6171, 6564, 6604, 6726, 6734, 6863, 6874, 7021, 7112, 7290, 7527, 7620, 7746, 7767, 7769, 8011, 8164, 8362, 8510, 8513, 8583, 8750, 8891, 9163, 9210, 9571, 9613, 9692, 9757, 9999, 10090, 10255, 10322, 10600, 10872, 10898, 10908, 11061, 11140, 11145, 11159, 11231, 11403, 11471, 11500, 11530, 11532, 11542, 11556, 11557, 11561, 11577, 11579, 11596, 11644, 11735, 11753, 11755, 11832, 11835, 11871, 11872, 11873, 11928, 11960, 11966, 11968, 11979, 11997, 12029, 12040, 12159, 12224, 12235, 12263, 12264, 12270, 12282, 12308, 12402, 12407, 12419, 12473, 12484, 12556, 12589, 12602, 12608, 12731, 12787, 12870, 12896, 12898, 12998, 13045, 13065, 1309

In [ ]:
# Testing the best model
#na_predictions = best_model.predict(test_final)
#test_final_df['victima_convive_agresor_pred'] = na_predictions

In [59]:
import numpy as np


Test final created


In [58]:
nsnc_indices.shape

(2977,)

In [56]:
X_nmds.shape

(17263, 2)

In [ ]:
# Preprocess dataset_b
llamados_v2 = pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v2.xlsx'))
llamados_v2.drop(['llamante_edad'], axis=1, inplace=True)
dataset_b = llamados_v2[~llamados_v2['victima_edad'].isnull()]

# Replace 'NS/NC' with NA and map 'SI' to 1, 'NO' to 0
dataset_b['victima_convive_agresor'] = dataset_b['victima_convive_agresor'].replace({'NS/NC': pd.NA})
dataset_b['victima_convive_agresor'] = dataset_b['victima_convive_agresor'].map({'SI': 1, 'NO': 0})

# Separate features and target
X = dataset_b.drop(['victima_convive_agresor'], axis=1)
y_previo = dataset_b['victima_convive_agresor']

# Identify NSNC indices and clean the target
nsnc_indices = y_previo[y_previo.isna()].index
y = y_previo.drop(nsnc_indices)

In [ ]:
# Testing the best model
na_predictions = best_model.predict(test_final)
test_final_df['victima_convive_agresor_pred'] = na_predictions